# **Training Gemma 3 Model for Russian to Chukchi Translation**

## **Imports**

In [1]:
!pip install bitsandbytes accelerate transformers peft datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
# !pip install -U flash-attn --no-build-isolation

In [3]:
import os
import shutil
import torch
import gc
import torch
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig, DataCollatorForSeq2Seq, TrainerCallback
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from huggingface_hub import notebook_login, create_repo, upload_folder
import pandas as pd
import json

In [4]:
notebook_login()

## **Connecting to Google Drive**

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
DRIVE_PATH = "/content/drive/MyDrive/gemma3_lora_rus_ckt"
os.makedirs(DRIVE_PATH, exist_ok=True)

## **GPU Configuration**

In [6]:
print(f"CUDA доступна: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA доступна: True
GPU: NVIDIA L4


In [7]:
device_map = "auto"
device_capability = torch.cuda.get_device_capability()[0]

## **Data Preparation**

In [11]:
# dataset = load_dataset("csv", data_files="ckt-ru_filtered.csv", sep=";")

df = pd.read_csv("ckt-ru_filtered.csv", sep=";")

# Converting to Dataset
dataset = Dataset.from_pandas(df)

In [12]:
dataset = dataset.rename_columns({
    "ru": "rus",
    "ckt": "ckt"
})

In [14]:
dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

### **Tokenizer Preparation**

In [15]:
model_name = 'google/gemma-3-1b-pt'
source_lang = 'rus'
target_lang = 'ckt'

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [17]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [40]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map=device_map,
    quantization_config=quant_config,
    # attn_implementation="flash_attention_2" if device_capability >= 8 else "sdpa",
    attn_implementation="eager",
    torch_dtype=torch.bfloat16,
)

In [41]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

## **LoRA Adapter Configuration**

In [42]:
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 1,490,944 || all params: 1,001,376,896 || trainable%: 0.1489


## **Preprocessing**

In [43]:
def generate_prompt(example):
    return f"Переведи с русского на чукотский: {example['rus']}"

In [44]:
def tokenize(example):
    # Create full text: prompt + translation
    prompt = generate_prompt(example)
    full_text = f"{prompt}\n{example['ckt']}"

    # Tokenize full text
    tokenized = tokenizer(
        full_text,
        truncation=True,
        max_length=512,
        return_tensors="pt",
    )

    # Calculate prompt length in tokens
    with tokenizer.as_target_tokenizer():
        prompt_ids = tokenizer(prompt)["input_ids"]

    input_length = len(prompt_ids)

    # Create loss mask (ignore prompt)
    labels = tokenized["input_ids"].clone()
    labels[:, :input_length] = -100

    return {
        "input_ids": tokenized["input_ids"][0],
        "attention_mask": tokenized["attention_mask"][0],
        "labels": labels[0]
    }

In [45]:
tokenized_train_dataset = train_dataset.map(tokenize, remove_columns=train_dataset.column_names)
tokenized_eval_dataset = eval_dataset.map(tokenize, remove_columns=eval_dataset.column_names)

Map:   0%|          | 0/53806 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13452 [00:00<?, ? examples/s]

In [46]:
tokenized_train_dataset.save_to_disk(f"{DRIVE_PATH}/train_dataset")
tokenized_eval_dataset.save_to_disk(f"{DRIVE_PATH}/eval_dataset")

Saving the dataset (0/1 shards):   0%|          | 0/53806 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/13452 [00:00<?, ? examples/s]

In [47]:
# Filtering out overly long examples
max_length = 512
tokenized_train_dataset = tokenized_train_dataset.filter(
    lambda x: len(x["input_ids"]) <= max_length
)
tokenized_eval_dataset = tokenized_eval_dataset.filter(
    lambda x: len(x["input_ids"]) <= max_length
)

Filter:   0%|          | 0/53806 [00:00<?, ? examples/s]

Filter:   0%|          | 0/13452 [00:00<?, ? examples/s]

## **Model Training**

Custom class for saving checkpoints to Google Drive

In [48]:
class DriveCheckpointCallback(TrainerCallback):
    def __init__(self, max_to_keep=2):
        self.max_to_keep = max_to_keep

    def on_save(self, args, state, control, **kwargs):
        if state.is_world_process_zero:
            checkpoint_path = os.path.join(args.output_dir, f"checkpoint-{state.global_step}")
            drive_checkpoint_path = os.path.join(DRIVE_PATH, f"checkpoint-{state.global_step}")

            try:
                if not os.path.exists(drive_checkpoint_path):
                    shutil.copytree(checkpoint_path, drive_checkpoint_path)
                    print(f"Checkpoint saved to Google Drive: {drive_checkpoint_path}")
                else:
                    print(f"Skipped copying: {drive_checkpoint_path} already exists.")
            except Exception as e:
                print(f"Copy error: {str(e)}")

            # Cleaning up old checkpoints
            checkpoints = [d for d in os.listdir(DRIVE_PATH) if d.startswith("checkpoint-")]
            checkpoints = sorted(checkpoints, key=lambda x: int(x.split("-")[-1]))

            if len(checkpoints) > self.max_to_keep:
                for ckpt in checkpoints[:-self.max_to_keep]:
                    path_to_remove = os.path.join(DRIVE_PATH, ckpt)
                    try:
                        shutil.rmtree(path_to_remove)
                        print(f"Removed old checkpoint: {path_to_remove}")
                    except Exception as e:
                        print(f"Deletion error {path_to_remove}: {e}")

Custom class for saving logs to Google Drive (for later visualization)

In [49]:
class LossHistoryCallback(TrainerCallback):
    def __init__(self, save_path):
        self.save_path = save_path
        self.loss_history = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return
        record = {"Step": state.global_step}
        if "loss" in logs:
            record["Training_Loss"] = logs["loss"]
        if "eval_loss" in logs:
            record["Validation_Loss"] = logs["eval_loss"]
        self.loss_history.append(record)

        # Saving to file
        with open(self.save_path, "w") as f:
            json.dump(self.loss_history, f, indent=2)

In [50]:
os.makedirs("/content/drive/MyDrive/gemma3_logs", exist_ok=True)

In [51]:
loss_json_path = "/content/drive/MyDrive/gemma3_logs/losses_gemma3_rus_ckt.json"
loss_callback = LossHistoryCallback(save_path=loss_json_path)

In [52]:
# Dynamic padding collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    padding=True,
    pad_to_multiple_of=8,
    return_tensors="pt"
)

In [53]:
training_args = TrainingArguments(
    output_dir=DRIVE_PATH,
    logging_dir=os.path.join(DRIVE_PATH, "logs"),
    logging_steps=50,
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    max_steps=1000,
    weight_decay=0.01,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    gradient_checkpointing=True,
    bf16=device_capability >= 8,   # Modern GPUs
    fp16=device_capability < 8,    # Older GPUs
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="none",
    group_by_length=True,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

In [54]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[loss_callback, DriveCheckpointCallback(max_to_keep=2)]
)

<ipython-input-54-3299213989>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [55]:
trainer.train()

Step,Training Loss,Validation Loss
200,3.422600,3.542935
400,3.173900,3.351145
600,3.119900,3.192377
800,3.054800,3.074218
1000,3.019800,3.054675


Пропущено копирование: /content/drive/MyDrive/gemma3_lora_rus_ckt/checkpoint-200 уже существует.
Пропущено копирование: /content/drive/MyDrive/gemma3_lora_rus_ckt/checkpoint-400 уже существует.
Пропущено копирование: /content/drive/MyDrive/gemma3_lora_rus_ckt/checkpoint-600 уже существует.
Пропущено копирование: /content/drive/MyDrive/gemma3_lora_rus_ckt/checkpoint-800 уже существует.
Пропущено копирование: /content/drive/MyDrive/gemma3_lora_rus_ckt/checkpoint-1000 уже существует.


TrainOutput(global_step=1000, training_loss=3.3069122619628906, metrics={'train_runtime': 7095.6338, 'train_samples_per_second': 1.127, 'train_steps_per_second': 0.141, 'total_flos': 1369478093008896.0, 'train_loss': 3.3069122619628906, 'epoch': 0.14868230308887484})

In [ ]:
# Memory cleanup
del trainer
torch.cuda.empty_cache()
gc.collect()

In [56]:
final_model_path = os.path.join(DRIVE_PATH, "final_model")
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

('/content/drive/MyDrive/gemma3_lora_rus_ckt/final_model/tokenizer_config.json',
 '/content/drive/MyDrive/gemma3_lora_rus_ckt/final_model/special_tokens_map.json',
 '/content/drive/MyDrive/gemma3_lora_rus_ckt/final_model/tokenizer.model',
 '/content/drive/MyDrive/gemma3_lora_rus_ckt/final_model/added_tokens.json',
 '/content/drive/MyDrive/gemma3_lora_rus_ckt/final_model/tokenizer.json')

In [57]:
# Weight merging for full model save
merged_model = model.merge_and_unload()
merged_model.save_pretrained(os.path.join(DRIVE_PATH, "merged_model"))

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


## **Model Testing**

In [7]:
# Load best model
model_path = os.path.join(DRIVE_PATH, "merged_model")
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [8]:
# Translation generation
test_prompt = 'Переведи с русского на чукотский: Привет, как дела?'
inputs = tokenizer(test_prompt, return_tensors="pt", padding=True).to(model.device)

In [9]:
model.eval()
with torch.no_grad():
    output_ids = model.generate(
        **inputs,
        max_new_tokens=50,
        num_beams=5,
        early_stopping=True,
        temperature=0.7,
        do_sample=True
    )

In [10]:
translation = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Translation result:", translation)

Translation result: Переведи с русского на чукотский: Привет, как дела?
Ынӈэ, ынӈэ, ынӈэ, ынӈэ?
Ынӈэ, ынӈэ, ынӈэ, ынӈэ?


## **Uploading Model to Hugging Face**

In [64]:
from huggingface_hub import HfApi

# Repository configuration
repo_name = "gemma3-1b-rus-ckt"
organization = "HSE-Chukchi-NLP"

# Creating repository
create_repo(f"{organization}/{repo_name}", repo_type="model")

# Uploading model
api = HfApi()
api.upload_folder(
    folder_path=os.path.join(DRIVE_PATH, "merged_model"),
    repo_id=f"{organization}/{repo_name}",
    repo_type="model"
)

print(f"Model uploaded: https://huggingface.co/{organization}/{repo_name}")

adapter_model.safetensors:   0%|          | 0.00/5.99M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.60G [00:00<?, ?B/s]

Модель загружена: https://huggingface.co/HSE-Chukchi-NLP/gemma3-1b-rus-ckt
